In [2]:
# Imports

import requests
import re
import time
import os
from bs4 import BeautifulSoup
import xml.sax
import pandas as pd

In [3]:
"""Scrapes https://www.bundestag.de/services/opendata for all XML Files of the 19th and 20th election period using selenium and chromedriver."""

"""from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--profile-directory=Profile2")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--headless")

url = "https://www.bundestag.de/services/opendata"
chromedriver = "/usr/bin/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver, options=chrome_options) #deprecation warning: pass Service object
driver.get(url)

# fügt alle gefundenen Links einer Liste hinzu
def add_to_link_list():
    soup_links = BeautifulSoup(driver.page_source, 'html.parser')
    regex = re.compile('.*XML .*')
    documents = soup_links.find_all(title=regex)
    for i in documents:
        link_list.append(i['href'])

for i in range(26):
    button = WebDriverWait(driver, 3).until(EC.visibility_of_element_located(
            #(By.XPATH, "//button[@type='button'][@class='slick-next slick-arrow']")))
    driver.execute_script("arguments[0].scrollIntoView();", button)
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)

#add_to_link_list()"""

'from selenium import webdriver\nfrom selenium.webdriver.chrome.options import Options\nfrom selenium.webdriver.common.keys import Keys\nfrom selenium.webdriver.common.by import By\nfrom selenium.webdriver.support.ui import WebDriverWait\nfrom selenium.webdriver.support import expected_conditions as EC\n\nchrome_options = webdriver.ChromeOptions()\nchrome_options.add_argument("--profile-directory=Profile2")\nchrome_options.add_argument("--disable-dev-shm-usage")\nchrome_options.add_argument(\'--incognito\')\nchrome_options.add_argument(\'--no-sandbox\')\nchrome_options.add_argument("--headless")\n\nurl = "https://www.bundestag.de/services/opendata"\nchromedriver = "/usr/bin/chromedriver"\nos.environ["webdriver.chrome.driver"] = chromedriver\n\ndriver = webdriver.Chrome(chromedriver, options=chrome_options) #deprecation warning: pass Service object\ndriver.get(url)\n\n# fügt alle gefundenen Links einer Liste hinzu\ndef add_to_link_list():\n    soup_links = BeautifulSoup(driver.page_sour

In [4]:
# neue Version ohne selenium

link_list = []

for i in range(3):
    cursor = "&offset=" + str(10*i)
    url_20_wahlperiode_stem = "https://www.bundestag.de/ajax/filterlist/de/services/opendata/866354-866354?limit=10&noFilterSet=true"
    url_20_wahlperiode = url_20_wahlperiode_stem + cursor
    req = requests.get(url_20_wahlperiode)
    soup_links = BeautifulSoup(req.content, 'html.parser')
    regex = re.compile('.*XML .*')
    documents = soup_links.find_all(title=regex)
    for i in documents:
        link_list.append(i['href'])

for i in range(24):
    cursor = "&offset=" + str(10*i)
    url_19_wahlperiode_stem = "https://www.bundestag.de/ajax/filterlist/de/services/opendata/543410-543410?limit=10&noFilterSet=true"
    url_19_wahlperiode = url_19_wahlperiode_stem + cursor
    req = requests.get(url_19_wahlperiode)
    soup_links = BeautifulSoup(req.content, 'html.parser')
    regex = re.compile('.*XML .*')
    documents = soup_links.find_all(title=regex)
    for i in documents:
        link_list.append(i['href'])     

# Liste mit vollständigen URLs
url_list = []
url_start = 'https://www.bundestag.de'

for i in link_list:
    url_list.append(url_start + i)

In [13]:
# Parser 
class SpeechHandler (xml.sax.ContentHandler):
    
    def __init__(self):
        self.CurrentData = ""
        self.p = ""
        self.tagesordnungspunkt = ""
        self.vorname = ""
        self.nachname = ""
        self.fraktion = ""
        self.redeID = ""
        self.startReden = False
        self.datum = ""
        

    def startElement(self, tag, attributes):

        self.CurrentData = tag
        
        if tag == "rede":
            self.redeID = attributes['id']
            self.startReden = True
            self.p = ""
        elif tag == "datum":
            self.datum = attributes['date']
    
    def endElement(self,tag):
        self.CurrentData = tag
        if self.CurrentData == 'rede':
            store.loc[len(store.index)] = [ self.datum, self.redeID, self.vorname, self.nachname, self.fraktion, self.p]
            self.p = ""
            self.fraktion = ""
            self.vorname = ""
            self.nachname = ""
    
    def characters (self, content):

        if self.CurrentData == "p" :
            self.p = self.p + content
        elif self.CurrentData == "vorname" and self.startReden and self.vorname == "":
            self.vorname = content
        elif self.CurrentData == "nachname" and self.startReden and self.nachname == "":
            self.nachname =  content 
        elif self.CurrentData == "fraktion" and self.startReden and self.fraktion == "":
            self.fraktion =  content

In [22]:
# parsing
count = 1
for file in url_list:
    # hier kann man jetzt alle Files einzeln parsen
    store = pd.DataFrame({ 'datum' : [''], 'redeID': [''], 'vorname': [''], 'nachname' : [''], 'fraktion': [''], 'inhalt' : ['']})
    parser  = xml.sax.make_parser()
    Handler = SpeechHandler()
    parser.setContentHandler(Handler)
    parser.parse(file)
    store['name'] = store['vorname'] + ' ' + store['nachname']
    
    # Clean speeches
    store['inhalt_neu'] = store['inhalt']
    alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
    store['inhalt_neu'] = store['inhalt_neu'].apply(alphanumeric)
    punctuation = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
    #store['inhalt_neu'] = store['inhalt_neu'].apply(punctuation)
    xa0 = lambda x: re.sub('\xa0', ' ', x)
    xad = lambda x: re.sub('\xad', ' ', x)
    store['inhalt_neu'] = store['inhalt_neu'].apply(xa0).apply(xad)
    dash = lambda x: re.sub('–', ' ', x)
    store['inhalt_neu'] = store['inhalt_neu'].apply(dash)
    lower = lambda x: x.lower()
    store['inhalt_neu'] = store['inhalt_neu'].apply(lower)
    hyphen_1 = lambda x: re.sub('“', ' ', x)
    hyphen_2 = lambda x: re.sub('„', ' ', x)
    store['inhalt_neu'] = store['inhalt_neu'].apply(hyphen_1).apply(hyphen_2)
    spaces = lambda x: ' '.join(x.split())
    store['inhalt_neu'] = store['inhalt_neu'].apply(spaces)
    data = store.drop(['inhalt'], axis=1)
    
    data.to_csv('bundestag ' + str(count) + '.csv')
    count = count + 1
 

        datum       redeID   vorname nachname fraktion             name  \
0                                                                         
1  24.03.2022  ID202500100    Robert   Habeck             Robert Habeck   
2  24.03.2022  ID202500200   Andreas     Jung  CDU/CSU     Andreas Jung   
3  24.03.2022  ID202500300     Frank    Junge      SPD      Frank Junge   
4  24.03.2022  ID202500400  Wolfgang   Wiehle      AfD  Wolfgang Wiehle   

                                          inhalt_neu  
0                                                     
1  frau präsidentin, sehr geehrte damen und herre...  
2  frau präsidentin! liebe kolleginnen und kolleg...  
3  frau präsidentin! liebe kolleginnen und kolleg...  
4  sehr geehrte frau präsidentin! kolleginnen und...  
        datum       redeID   vorname          nachname               fraktion  \
0                                                                               
1  25.03.2022  ID202600100  Hubertus              Heil   

In [13]:
#driver.quit()